In [1]:
!pip install transformers
!pip install datasets

DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\argparse-1.4.0-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\collection-0.1.6-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\configparser-7.1.0-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\l

DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\argparse-1.4.0-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\collection-0.1.6-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\local\programs\python\python313\lib\site-packages\configparser-7.1.0-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\pc\appdata\l

# Assignment 2: RNN and NER
Welcome to the second assignment of CSE-476! In this project, you will be asked to implement a functional RNN using Pytorch, and then use it to do the same task as last time: NER. Hopefully, you will observe a much better perofermance with RNNs.

## Task 1: Implement the RNN

Your first task is to implement the missing part of the RNN.

In [26]:
import torch
import torch.nn as nn
import math

class SimpleRNNCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleRNNCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        '''
        TODO: Create all the necessary weights here.
        Recall for RNN, you need
        - a weight matrix from input to hidden (W)
        - a weight matrix from hidden to hidden (U)
        - a bias term b of size hidden, so that we have (Wx+Uh+b) before activation
        '''
        self.weights_ih = nn.Linear(self.input_dim, self.hidden_dim, bias=False)#nn.Parameter(torch.empty(self.input_dim, self.hidden_dim))
        self.weights_hh = nn.Linear(self.hidden_dim, self.hidden_dim, bias=False)#nn.Parameter(torch.empty(self.hidden_dim, self.hidden_dim))
        self.bias = nn.Parameter(torch.zeros(self.hidden_dim))
        # Weights defintions below

        # Call weight initialization function
        self.init_weights()

    def init_weights(self):
        '''
        TODO: initialize the weights W/U with
        https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_uniform_
        Note as this function takes a tensor, you need to pass in W.weights,
        if you initialize W with nn.Linear().
        '''
        nn.init.kaiming_uniform_(self.weights_ih.weight)
        nn.init.kaiming_uniform_(self.weights_hh.weight)

    def forward(self, x_cur, hidden_prev):
        '''
        TODO: compute the forward pass using torch.tanh() as the activation function.
        You should add the transformed input (size hidden_dim) and the transformed previous weights (size hidden_dim)
        and the bias term b into a size hidden_dim vector, as the input to tanh().
        i.e., (Wx+Uh+b)
        Return the current hidden state.
        '''
        h_cur = torch.tanh(self.weights_ih(x_cur) + self.weights_hh(hidden_prev) + self.bias)
        return h_cur


# This class is a wrapper to the RNN cell to make it work on any output dims.
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn_cell = SimpleRNNCell(input_dim, hidden_dim)
        '''
        TODO: you need to define some weights here to map the hidden size to output size
        This is the V matrix as defined in the slides
        '''
        self.V = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # The last part of x is the hidden size, which is not needed
        batch_size, seq_len, _ = x.size()

        '''
        TODO: finish the implementation of the RNN forward call.
        You need to
        - 1) keep track of what if the current hidden state.
        - 2) implement a for loop to acquire all the hidden states for each input
        - 3) keep all the hidden states from each timestamp
        - 4) compute the output y using V and the hidden states (note, you can do it for each timestamp, or together in one call)
        Return all the predicted scores y_scores.
        If you print y_scores.shape, it should be (batch_size, seq_len, output_dim)
        Meaning that you are giving a output score/probability for all possible labels at all timestamp/positions, in all the sequences in the batch.
        Hint: you may want to consider initializing first hidden state with zero values.
        '''
        cur_hid = torch.zeros(batch_size, self.hidden_dim)
        states = list()
        
        for i in range(seq_len):
            x_cur = x[:, i, :]
            cur_hid = self.rnn_cell(x_cur, cur_hid)
            states.append(cur_hid)
            
        states_stack = torch.stack(states, dim=1)
        y_scores = self.V(states_stack)
        
        return y_scores

In [29]:
# This is a wrapper class for using RNN on NER.
# A minimum working version is given, but you can (and should) customize it to make the performances better.
class SimpleRNNForNER(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=64, hidden_dim=128, output_dim=256, dropout=0.2, num_layers=3):
        super(SimpleRNNForNER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        # Note here that we are using "hidden_dim" as the output_dim for our RNN class,
        # and then project that to the number of NER labels (tagset_size)
        # Is it the best thing to do? Think about and play with it.
        self.rnn_layers = nn.ModuleList()
        self.output_dim = hidden_dim*2
        self.rnn = SimpleRNN(embedding_dim, hidden_dim, hidden_dim)
        self.rnn_layers.append(self.rnn)

        for i in range(num_layers-2):
            embedding_dim = hidden_dim
            #hidden_dim *= 2
            #output_dim = hidden_dim*2
            #self.rnn_layers.append(SimpleRNN(hidden_dim, hidden_dim, hidden_dim))
            self.rnn_layers.append(SimpleRNN(embedding_dim, hidden_dim, hidden_dim))

        #output_dim = hidden_dim*2
        self.rnn_layers.append(SimpleRNN(hidden_dim, hidden_dim, output_dim))
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(output_dim, tagset_size)

    def forward(self, x):
        x = self.embedding(x)
        #x = self.embedding_dropout(x)

        for i, rnn_layer in enumerate(self.rnn_layers):
            x = rnn_layer(x)
            #if i < len(self.rnn_layers) - 1:
            #    x = self.dropout(x)
                
        x = self.dropout(x)
        tag_scores = self.fc(x)

        return tag_scores

## Task 2: Applying SimpleRNNForNER to NER.
Congratulations of getting this far! Now we have the model for NER, let's use it!

In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from torch.utils.data.dataloader import default_collate

# We are providing an implementation to load the NER dataset.
# However, you should still read it over carefully to understand how it works.
class NERDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def extract_ner_data():
    # Load the CoNLL 2003 dataset
    datasets = load_dataset('conll2003', trust_remote_code=True)

    # Load a pre-trained tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Tokenize and encode the datasets
    def encode_tags(tags, encodings, tag2id):
        labels = [[tag for tag in doc] for doc in tags]
        encoded_labels = []
        for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
            # create an empty array of -100
            doc_enc_labels = np.ones(len(doc_offset), dtype=int) * -100
            arr_offset = np.array(doc_offset)

            # set labels whose first offset position is 0 and the second is not 0
            doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)] = doc_labels
            encoded_labels.append(doc_enc_labels.tolist())

        return encoded_labels

    tag2id = {tag: i for i, tag in enumerate(datasets['train'].features['ner_tags'].feature.names)}
    id2tag = {i: tag for tag, i in tag2id.items()}

    # Tokenize the input sentences and encode labels
    train_encodings = tokenizer(datasets['train']['tokens'], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    train_labels = encode_tags(datasets['train']['ner_tags'], train_encodings, tag2id)

    val_encodings = tokenizer(datasets['validation']['tokens'], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    val_labels = encode_tags(datasets['validation']['ner_tags'], val_encodings, tag2id)

    # Create datasets
    train_dataset = NERDataset(train_encodings, train_labels)
    val_dataset = NERDataset(val_encodings, val_labels)

    return train_dataset, val_dataset, tag2id, id2tag


def train(model, train_dataset, val_dataset, epochs=3, batch_size=32):
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    '''
    TODO: You need to define the optimizer and the criterions here.
    You should use
    - torch.optim.Adam() as the optimizer
    - nn.CrossEntropyLoss as the criterion
    Hint: we don't want to penalize the model on padded labels (-100).
    nn.CrossEntropyLoss has "ignore_index" argument that you can utilze.
    '''
    optimizer = torch.optim.Adam(params=model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=-100)

    # Id to labels mapping
    id_to_labels = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
            # inputs here should be in a ready-shape as input parameter to SimpleRNNForNER's forward() call.
            # in other words, you should be able to call model(inputs) without errors if your model is correctly implemented.
            inputs, labels = batch['input_ids'], batch['labels']
            '''
            TODO: implement the missing parts in this training loop.
            - remember zero_grad()?
            - remember backward()?
            - remember step()?
            '''
            optimizer.zero_grad()
            res = model(inputs)
            x, y, z = res.size()

            res_tensor = res.view(-1, z)
            labels_tensor = labels.view(-1)
            
            loss = criterion(res_tensor, labels_tensor)
            loss.backward()
            optimizer.step()
            # note this loss should be defined in your implementation above.
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")

        '''
        TODO: what do we need to do to put model in eval mode?
        '''
        model.eval()
        total_val_loss = 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} - Evaluation"):
                inputs, labels = batch['input_ids'], batch['labels']
                outputs = model(inputs)
                loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
                total_val_loss += loss.item()

                preds = torch.argmax(outputs, dim=-1)
                all_preds.extend(preds.view(-1).cpu().numpy())
                all_labels.extend(labels.view(-1).cpu().numpy())

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Epoch {epoch+1}, Val Loss: {avg_val_loss:.4f}")

        '''
        TODO: We are providing the F1 calculation for each label in the NER below.
        However, when you are iterating the model design to improve the performances,
        it is ideal to have a single number performance. To do this, you will implement
        Macro-F1 score, which is simply the average of all F1 scores from all labels.
        Return this score is encouraged, so you can observe improvements later.
        '''
        label_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0})

        for true_label, pred_label in zip(all_labels, all_preds):
            if true_label == -100:
                continue  # Ignore padded tokens

            for label_id in id_to_labels.keys():
                if true_label == label_id:
                    if pred_label == true_label:
                        label_stats[label_id]['tp'] += 1
                    else:
                        label_stats[label_id]['fn'] += 1
                elif pred_label == label_id:
                    label_stats[label_id]['fp'] += 1
        macro_f1 = list()
        for label_id, stats in label_stats.items():
            precision = stats['tp'] / (stats['tp'] + stats['fp']) if (stats['tp'] + stats['fp']) > 0 else 0.0
            recall = stats['tp'] / (stats['tp'] + stats['fn']) if (stats['tp'] + stats['fn']) > 0 else 0.0
            f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

            label_name = id_to_labels[label_id]
            macro_f1.append(f1)
            print(f"Label '{label_name}': Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
        print(f"Macro F1: {sum(macro_f1)/len(macro_f1)}")

## Task 3: Test your NER model and improve

You should now have a working NER model and training/eval pipeline. Your job in this task is to run the model, observe the performances, and make improvements. We first provide you with a simple example script to train and test your current model.

In [30]:
# We first provide you with a minimum code to make the entire train/eval pipeline working
# so you can observe how it works and what the outputs are.
train_data, eval_data, _, _ = extract_ner_data()
model = SimpleRNNForNER(50000, 9)
train(model, train_data, eval_data)

Epoch 1 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:09<00:00,  3.39it/s]


Epoch 1, Train Loss: 0.5929


Epoch 1 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:05<00:00, 17.86it/s]


Epoch 1, Val Loss: 0.4760
Label 'O': Precision: 0.8901, Recall: 0.9860, F1 Score: 0.9356
Label 'B-ORG': Precision: 0.2915, Recall: 0.1872, F1 Score: 0.2280
Label 'B-LOC': Precision: 0.5589, Recall: 0.4807, F1 Score: 0.5168
Label 'B-MISC': Precision: 0.3133, Recall: 0.0510, F1 Score: 0.0877
Label 'I-MISC': Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Label 'B-PER': Precision: 0.7988, Recall: 0.2845, F1 Score: 0.4195
Label 'I-PER': Precision: 0.8439, Recall: 0.3351, F1 Score: 0.4797
Label 'I-ORG': Precision: 0.3609, Recall: 0.0639, F1 Score: 0.1086
Label 'I-LOC': Precision: 0.7010, Recall: 0.2646, F1 Score: 0.3842
Macro F1: 0.3511279164074805


Epoch 2 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:05<00:00,  3.49it/s]


...

Epoch 2, Val Loss: 0.3771
Label 'O': Precision: 0.9403, Recall: 0.9772, F1 Score: 0.9584
Label 'B-PER': Precision: 0.7357, Recall: 0.4245, F1 Score: 0.5384
Label 'B-ORG': Precision: 0.3448, Recall: 0.3296, F1 Score: 0.3370
Label 'I-ORG': Precision: 0.3034, Recall: 0.4407, F1 Score: 0.3594
Label 'I-LOC': Precision: 0.4570, Recall: 0.5370, F1 Score: 0.4937
Label 'B-LOC': Precision: 0.6673, Recall: 0.5732, F1 Score: 0.6167
Label 'B-MISC': Precision: 0.5477, Recall: 0.2614, F1 Score: 0.3539
Label 'I-MISC': Precision: 0.5730, Recall: 0.2948, F1 Score: 0.3893
Label 'I-PER': Precision: 0.7709, Recall: 0.5845, F1 Score: 0.6649
Macro F1: 0.523524735566532


Epoch 3 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:09<00:00,  3.38it/s]


Epoch 3, Train Loss: 0.2769


Epoch 3 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:05<00:00, 19.52it/s]


Epoch 3, Val Loss: 0.3232
Label 'O': Precision: 0.9576, Recall: 0.9773, F1 Score: 0.9673
Label 'B-ORG': Precision: 0.3856, Recall: 0.3781, F1 Score: 0.3818
Label 'B-LOC': Precision: 0.6454, Recall: 0.6935, F1 Score: 0.6686
Label 'I-ORG': Precision: 0.3940, Recall: 0.5965, F1 Score: 0.4746
Label 'I-MISC': Precision: 0.5353, Recall: 0.3728, F1 Score: 0.4395
Label 'B-MISC': Precision: 0.6116, Recall: 0.4338, F1 Score: 0.5076
Label 'B-PER': Precision: 0.7260, Recall: 0.4805, F1 Score: 0.5782
Label 'I-PER': Precision: 0.8103, Recall: 0.6373, F1 Score: 0.7135
Label 'I-LOC': Precision: 0.7134, Recall: 0.4358, F1 Score: 0.5411
Macro F1: 0.585801503699482


Then, you need to write some experiment code to provide at least two conclusions on what factor works, or not (negative conclusions are also valid conclusions). The conclusions do not need to be extremely extensive.

The conclusions you make can be towards studying the differences of
- 1) learning rates
- 2) hidden dim
- 3) different activations or more non-linear activations (e.g., in SimpleRNN)
- 4) more hidden layers
- 5) different weight initialization methods
- 6) ... (anything else as long as it makes sense)

In [ ]:
'''
TODO: Write your own code to experiment different factors.
You are free to change the parameter/function call/model structures in the RNN class, as long as
- 1) It's still a vanilla RNN (not its variant such as LSTM)
- 2) Our provided minimum running script above works, because we will use it for grading.
- 3) You do not use any Pytorch built-in models.
'''


### Dropout testing

In [31]:
train_data, eval_data, _, _ = extract_ner_data()
model = SimpleRNNForNER(50000, 9, dropout=0.4)
train(model, train_data, eval_data)

Epoch 1 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:32<00:00,  2.89it/s]


Epoch 1, Train Loss: 0.6088


Epoch 1 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:07<00:00, 14.22it/s]


Epoch 1, Val Loss: 0.4873
Label 'O': Precision: 0.8974, Recall: 0.9851, F1 Score: 0.9392
Label 'B-ORG': Precision: 0.3826, Recall: 0.0984, F1 Score: 0.1566
Label 'B-LOC': Precision: 0.4778, Recall: 0.5389, F1 Score: 0.5065
Label 'B-MISC': Precision: 0.2778, Recall: 0.0054, F1 Score: 0.0106
Label 'I-PER': Precision: 0.6130, Recall: 0.4836, F1 Score: 0.5406
Label 'I-MISC': Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Label 'B-PER': Precision: 0.7496, Recall: 0.2763, F1 Score: 0.4038
Label 'I-ORG': Precision: 0.3800, Recall: 0.0506, F1 Score: 0.0893
Label 'I-LOC': Precision: 0.4778, Recall: 0.3346, F1 Score: 0.3936
Macro F1: 0.33780645093102957


Epoch 2 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:37<00:00,  2.78it/s]


...

Epoch 2, Val Loss: 0.3847
Label 'O': Precision: 0.9369, Recall: 0.9782, F1 Score: 0.9571
Label 'B-ORG': Precision: 0.3424, Recall: 0.3945, F1 Score: 0.3666
Label 'B-LOC': Precision: 0.7333, Recall: 0.4894, F1 Score: 0.5870
Label 'I-ORG': Precision: 0.3206, Recall: 0.3249, F1 Score: 0.3228
Label 'B-MISC': Precision: 0.5490, Recall: 0.0911, F1 Score: 0.1563
Label 'I-PER': Precision: 0.6594, Recall: 0.6473, F1 Score: 0.6533
Label 'I-MISC': Precision: 0.7532, Recall: 0.1676, F1 Score: 0.2742
Label 'B-PER': Precision: 0.5869, Recall: 0.4929, F1 Score: 0.5359
Label 'I-LOC': Precision: 0.7109, Recall: 0.3541, F1 Score: 0.4727
Macro F1: 0.4806479782427064


Epoch 3 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:41<00:00,  2.71it/s]


Epoch 3, Train Loss: 0.2804


Epoch 3 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:06<00:00, 14.99it/s]


Epoch 3, Val Loss: 0.3331
Label 'O': Precision: 0.9518, Recall: 0.9806, F1 Score: 0.9660
Label 'B-ORG': Precision: 0.3606, Recall: 0.5101, F1 Score: 0.4225
Label 'B-LOC': Precision: 0.7801, Recall: 0.5117, F1 Score: 0.6180
Label 'I-ORG': Precision: 0.3907, Recall: 0.4834, F1 Score: 0.4321
Label 'B-MISC': Precision: 0.5083, Recall: 0.4306, F1 Score: 0.4662
Label 'I-MISC': Precision: 0.6796, Recall: 0.4046, F1 Score: 0.5072
Label 'B-PER': Precision: 0.7182, Recall: 0.4566, F1 Score: 0.5582
Label 'I-PER': Precision: 0.7992, Recall: 0.5998, F1 Score: 0.6853
Label 'I-LOC': Precision: 0.7464, Recall: 0.4008, F1 Score: 0.5215
Macro F1: 0.5752457142193325


In [32]:
train_data, eval_data, _, _ = extract_ner_data()
model = SimpleRNNForNER(50000, 9, dropout=0.1)
train(model, train_data, eval_data)

Epoch 1 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:13<00:00,  3.28it/s]


Epoch 1, Train Loss: 0.5912


Epoch 1 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:05<00:00, 18.78it/s]


Epoch 1, Val Loss: 0.5008
Label 'O': Precision: 0.8957, Recall: 0.9781, F1 Score: 0.9351
Label 'B-ORG': Precision: 0.2818, Recall: 0.3236, F1 Score: 0.3013
Label 'I-ORG': Precision: 0.2312, Recall: 0.1065, F1 Score: 0.1459
Label 'B-LOC': Precision: 0.5330, Recall: 0.4132, F1 Score: 0.4655
Label 'B-MISC': Precision: 0.2857, Recall: 0.0043, F1 Score: 0.0085
Label 'I-MISC': Precision: 0.2500, Recall: 0.0029, F1 Score: 0.0057
Label 'B-PER': Precision: 0.7711, Recall: 0.2524, F1 Score: 0.3804
Label 'I-PER': Precision: 0.8004, Recall: 0.2946, F1 Score: 0.4306
Label 'I-LOC': Precision: 0.3682, Recall: 0.3696, F1 Score: 0.3689
Macro F1: 0.33799470186830544


Epoch 2 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:24<00:00,  3.04it/s]


...

Epoch 2, Val Loss: 0.3968
Label 'O': Precision: 0.9141, Recall: 0.9883, F1 Score: 0.9498
Label 'B-ORG': Precision: 0.4037, Recall: 0.2409, F1 Score: 0.3017
Label 'I-ORG': Precision: 0.5154, Recall: 0.1784, F1 Score: 0.2651
Label 'B-LOC': Precision: 0.6116, Recall: 0.5966, F1 Score: 0.6040
Label 'B-MISC': Precision: 0.4643, Recall: 0.1410, F1 Score: 0.2163
Label 'I-MISC': Precision: 0.7463, Recall: 0.2890, F1 Score: 0.4167
Label 'B-PER': Precision: 0.7661, Recall: 0.3876, F1 Score: 0.5148
Label 'I-PER': Precision: 0.8231, Recall: 0.4912, F1 Score: 0.6152
Label 'I-LOC': Precision: 0.6772, Recall: 0.4163, F1 Score: 0.5157
Macro F1: 0.48880485782314537


Epoch 3 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [02:21<00:00,  3.09it/s]


Epoch 3, Train Loss: 0.2800


Epoch 3 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:05<00:00, 19.08it/s]


Epoch 3, Val Loss: 0.3373
Label 'O': Precision: 0.9428, Recall: 0.9828, F1 Score: 0.9624
Label 'B-ORG': Precision: 0.4505, Recall: 0.3669, F1 Score: 0.4044
Label 'B-LOC': Precision: 0.6696, Recall: 0.6146, F1 Score: 0.6409
Label 'I-MISC': Precision: 0.7107, Recall: 0.4046, F1 Score: 0.5157
Label 'B-MISC': Precision: 0.4784, Recall: 0.3970, F1 Score: 0.4339
Label 'I-ORG': Precision: 0.4893, Recall: 0.4554, F1 Score: 0.4717
Label 'B-PER': Precision: 0.7286, Recall: 0.4533, F1 Score: 0.5589
Label 'I-PER': Precision: 0.7780, Recall: 0.6060, F1 Score: 0.6813
Label 'I-LOC': Precision: 0.7312, Recall: 0.5292, F1 Score: 0.6140
Macro F1: 0.5870265223279222


### Hidden dim testing

In [33]:
train_data, eval_data, _, _ = extract_ner_data()
model = SimpleRNNForNER(50000, 9, hidden_dim=64)
train(model, train_data, eval_data)

Epoch 1 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [01:32<00:00,  4.77it/s]


Epoch 1, Train Loss: 0.6298


Epoch 1 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:04<00:00, 24.96it/s]


Epoch 1, Val Loss: 0.5196
Label 'O': Precision: 0.8792, Recall: 0.9873, F1 Score: 0.9301
Label 'B-ORG': Precision: 0.2780, Recall: 0.1999, F1 Score: 0.2325
Label 'B-LOC': Precision: 0.6654, Recall: 0.2771, F1 Score: 0.3912
Label 'B-MISC': Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Label 'I-PER': Precision: 0.5512, Recall: 0.2800, F1 Score: 0.3714
Label 'I-MISC': Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Label 'B-PER': Precision: 0.6081, Recall: 0.2611, F1 Score: 0.3654
Label 'I-LOC': Precision: 0.5865, Recall: 0.2374, F1 Score: 0.3380
Label 'I-ORG': Precision: 0.2857, Recall: 0.0213, F1 Score: 0.0397
Macro F1: 0.2964748819088397


Epoch 2 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [01:35<00:00,  4.61it/s]


...

Epoch 2, Val Loss: 0.4095
Label 'O': Precision: 0.9281, Recall: 0.9806, F1 Score: 0.9536
Label 'B-ORG': Precision: 0.3092, Recall: 0.2893, F1 Score: 0.2989
Label 'B-LOC': Precision: 0.5874, Recall: 0.5961, F1 Score: 0.5917
Label 'B-MISC': Precision: 0.5167, Recall: 0.0336, F1 Score: 0.0631
Label 'I-ORG': Precision: 0.3135, Recall: 0.2876, F1 Score: 0.3000
Label 'I-MISC': Precision: 0.8000, Recall: 0.1272, F1 Score: 0.2195
Label 'B-PER': Precision: 0.6356, Recall: 0.3882, F1 Score: 0.4820
Label 'I-PER': Precision: 0.7070, Recall: 0.5318, F1 Score: 0.6070
Label 'I-LOC': Precision: 0.6346, Recall: 0.3852, F1 Score: 0.4794
Macro F1: 0.4439180407869578


Epoch 3 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [01:35<00:00,  4.60it/s]


Epoch 3, Train Loss: 0.3042


Epoch 3 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:04<00:00, 21.17it/s]


Epoch 3, Val Loss: 0.3450
Label 'O': Precision: 0.9514, Recall: 0.9788, F1 Score: 0.9649
Label 'B-ORG': Precision: 0.3627, Recall: 0.4079, F1 Score: 0.3840
Label 'B-LOC': Precision: 0.6826, Recall: 0.6462, F1 Score: 0.6639
Label 'B-PER': Precision: 0.6869, Recall: 0.4609, F1 Score: 0.5517
Label 'B-MISC': Precision: 0.4490, Recall: 0.2386, F1 Score: 0.3116
Label 'I-LOC': Precision: 0.5556, Recall: 0.4475, F1 Score: 0.4957
Label 'I-MISC': Precision: 0.5370, Recall: 0.2514, F1 Score: 0.3425
Label 'I-PER': Precision: 0.7751, Recall: 0.6327, F1 Score: 0.6967
Label 'I-ORG': Precision: 0.3472, Recall: 0.4447, F1 Score: 0.3900
Macro F1: 0.5334315240717614


In [34]:
train_data, eval_data, _, _ = extract_ner_data()
model = SimpleRNNForNER(50000, 9, hidden_dim=256)
train(model, train_data, eval_data)

Epoch 1 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [05:19<00:00,  1.37it/s]


Epoch 1, Train Loss: 0.5681


Epoch 1 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:09<00:00, 11.27it/s]


Epoch 1, Val Loss: 0.4530
Label 'O': Precision: 0.9028, Recall: 0.9805, F1 Score: 0.9401
Label 'B-ORG': Precision: 0.3282, Recall: 0.2677, F1 Score: 0.2949
Label 'B-LOC': Precision: 0.5772, Recall: 0.5024, F1 Score: 0.5373
Label 'I-ORG': Precision: 0.3955, Recall: 0.0932, F1 Score: 0.1509
Label 'I-LOC': Precision: 0.7059, Recall: 0.4202, F1 Score: 0.5268
Label 'B-MISC': Precision: 0.3460, Recall: 0.0987, F1 Score: 0.1536
Label 'I-MISC': Precision: 0.7979, Recall: 0.2168, F1 Score: 0.3409
Label 'B-PER': Precision: 0.7766, Recall: 0.3208, F1 Score: 0.4541
Label 'I-PER': Precision: 0.7506, Recall: 0.4491, F1 Score: 0.5620
Macro F1: 0.4400530260381648


Epoch 2 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [05:32<00:00,  1.32it/s]


...

Epoch 2, Val Loss: 0.3676
Label 'O': Precision: 0.9307, Recall: 0.9827, F1 Score: 0.9560
Label 'B-ORG': Precision: 0.4486, Recall: 0.1984, F1 Score: 0.2751
Label 'B-LOC': Precision: 0.5940, Recall: 0.6347, F1 Score: 0.6137
Label 'I-ORG': Precision: 0.5040, Recall: 0.2517, F1 Score: 0.3357
Label 'B-MISC': Precision: 0.4593, Recall: 0.3059, F1 Score: 0.3672
Label 'I-MISC': Precision: 0.8456, Recall: 0.3324, F1 Score: 0.4772
Label 'B-PER': Precision: 0.7403, Recall: 0.4131, F1 Score: 0.5303
Label 'I-PER': Precision: 0.6548, Recall: 0.6764, F1 Score: 0.6654
Label 'I-LOC': Precision: 0.7452, Recall: 0.4553, F1 Score: 0.5652
Macro F1: 0.5317509963674997


Epoch 3 - Training: 100%|████████████████████████████████████████████████████████████| 439/439 [05:27<00:00,  1.34it/s]


Epoch 3, Train Loss: 0.2612


Epoch 3 - Evaluation: 100%|██████████████████████████████████████████████████████████| 102/102 [00:09<00:00, 10.33it/s]


Epoch 3, Val Loss: 0.3191
Label 'O': Precision: 0.9560, Recall: 0.9762, F1 Score: 0.9660
Label 'B-ORG': Precision: 0.4877, Recall: 0.3400, F1 Score: 0.4007
Label 'B-LOC': Precision: 0.5822, Recall: 0.7496, F1 Score: 0.6554
Label 'I-ORG': Precision: 0.4928, Recall: 0.4554, F1 Score: 0.4734
Label 'I-LOC': Precision: 0.7402, Recall: 0.5875, F1 Score: 0.6551
Label 'B-MISC': Precision: 0.6105, Recall: 0.3774, F1 Score: 0.4665
Label 'I-MISC': Precision: 0.7812, Recall: 0.4335, F1 Score: 0.5576
Label 'B-PER': Precision: 0.7056, Recall: 0.5114, F1 Score: 0.5930
Label 'I-PER': Precision: 0.6908, Recall: 0.7437, F1 Score: 0.7163
Macro F1: 0.609331173843403


## Submission (Task 1&2)
You will submit this Notebook with intermediate running outputs. If you made changes to the RNN classes, you can keep it as is, no need to revert back to a vanilla version.
## Submission (Task 2)
Write here on what are the two conclusions you found above? ~50 words (within 100) for each conclusion. Make sure your conclusion involves quantitative evidences from the Macro F1 score.

Basically I checked the effect hidden layers and dropout parameters have on the model's performance. In the modified architecture I implemented a single dropout before the final linear layer. The reasoning is to control the complexity of the model and attenuating possible overfitting problems. In our case, though, model is not too complex and increasing the dropout results in a slight drop in performance. Hidden layers, in their turn, increase the complexity of the model and, as results tell us, increase the performance by a considerate amount (~ 7% difference). The model should perform better with complexity increase.